In [25]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
from sklearn.pipeline import Pipeline
import pkg_resources
import glob
from sklearn.preprocessing import StandardScaler

In [26]:
file_path = {
    "2013_Targa_Sixty_Six": "data/2013_Targa_Sixty_Six",
    "2014_Targa_Sixty_Six": "data/2014_Targa_Sixty_Six",
    "2013_Laguna_Seca": "data/2013_Laguna_Seca",
}

dflist = []

for i, fname in enumerate(sorted(glob.glob("data/2013_Targa_Sixty_Six" + "/*.csv"))):
    dflist.append(pd.read_csv(fname, sep=","))

testlist = [dflist[0][:5], dflist[1][:5]]

In [31]:
testlist

[   sideSlip  vxCG  vyCG  time  longitude  latitude  engineSpeed  \
 0       0.0   0.0  -0.0  0.00        NaN       NaN          210   
 1       0.0   0.0  -0.0  0.05        NaN       NaN          210   
 2       0.0   0.0  -0.0  0.10        NaN       NaN          210   
 3       0.0   0.0  -0.0  0.15        NaN       NaN          210   
 4       0.0   0.0  -0.0  0.20        NaN       NaN          210   
 
    handwheelAngle  throttle  brake  axCG  ayCG  yawRate  chassisAccelFL  \
 0           -22.3       0.5    0.0  0.04 -0.20    -0.21            9.56   
 1           -22.2       0.5    0.0 -0.09 -0.03    -0.03            9.58   
 2           -22.3       0.5    0.0 -0.03 -0.10     0.11            9.60   
 3           -22.2       0.5    0.0 -0.02 -0.09     0.11            9.61   
 4           -22.2       0.5    0.0  0.01 -0.14    -0.35            9.59   
 
    chassisAccelFR  chassisAccelRL  chassisAccelRR  
 0            9.61            9.01            9.14  
 1            9.64        

In [27]:
class concatDataFrames(BaseEstimator,TransformerMixin):

    def __init__(self, df_id):
        self.df_id = df_id

    def fit(self, dflist, y = None):
        return self
    
    def transform(self, dflist):
        df_concat = pd.DataFrame([])
        for i, df in enumerate(dflist):
            df[self.df_id] = i
            df_concat = pd.concat([df_concat, df])
        return df_concat

In [44]:
class separateDataFrames(BaseEstimator,TransformerMixin):

    def __init__(self, df_id):
        self.df_id = df_id

    def fit(self, dflist, y = None):
        return self
    
    def transform(self, df):
        return [g.drop(columns=[self.df_id]) for _,g in df.groupby(self.df_id)]

In [45]:
pip = Pipeline([("test",TestImputer("vxCG"))])
col = pip.transform(testlist)

In [46]:
testlist[0]

,sideSlip,vxCG,vyCG,time,longitude,latitude,engineSpeed,handwheelAngle,throttle,brake,axCG,ayCG,yawRate,chassisAccelFL,chassisAccelFR,chassisAccelRL,chassisAccelRR,id
0,0.0,0.0,-0.0,0.00,NaN,NaN,210,-22.3,0.5,0.0,0.04,-0.20,-0.21,9.56,9.61,9.01,9.14,0
1,0.0,0.0,-0.0,0.05,NaN,NaN,210,-22.2,0.5,0.0,-0.09,-0.03,-0.03,9.58,9.64,9.16,9.25,0
2,0.0,0.0,-0.0,0.10,NaN,NaN,210,-22.3,0.5,0.0,-0.03,-0.10,0.11,9.60,9.61,9.07,9.14,0
3,0.0,0.0,-0.0,0.15,NaN,NaN,210,-22.2,0.5,0.0,-0.02,-0.09,0.11,9.61,9.62,9.01,9.10,0
4,0.0,0.0,-0.0,0.20,NaN,NaN,210,-22.2,0.5,0.0,0.01,-0.14,-0.35,9.59,9.59,9.06,9.20,0


In [47]:
pip = Pipeline([("test",concatDataFrames(df_id = "id"))])
col = pip.transform(testlist)


/var/folders/dp/4gxlxdsj3s5gqzxbyb19tvl00000gn/T/ipykernel_5458/47998008.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[self.df_id] = i


In [48]:
col

,sideSlip,vxCG,vyCG,time,longitude,latitude,engineSpeed,handwheelAngle,throttle,brake,axCG,ayCG,yawRate,chassisAccelFL,chassisAccelFR,chassisAccelRL,chassisAccelRR,id
0,0.0,0.00,-0.00,0.00,NaN,NaN,210,-22.3,0.5,0.0,0.04,-0.20,-0.21,9.56,9.61,9.01,9.14,0
1,0.0,0.00,-0.00,0.05,NaN,NaN,210,-22.2,0.5,0.0,-0.09,-0.03,-0.03,9.58,9.64,9.16,9.25,0
2,0.0,0.00,-0.00,0.10,NaN,NaN,210,-22.3,0.5,0.0,-0.03,-0.10,0.11,9.60,9.61,9.07,9.14,0
3,0.0,0.00,-0.00,0.15,NaN,NaN,210,-22.2,0.5,0.0,-0.02,-0.09,0.11,9.61,9.62,9.01,9.10,0
4,0.0,0.00,-0.00,0.20,NaN,NaN,210,-22.2,0.5,0.0,0.01,-0.14,-0.35,9.59,9.59,9.06,9.20,0
0,0.0,-0.01,0.00,0.00,-80.306057,26.920794,2178,-13.6,2.2,0.0,0.22,0.57,-0.06,10.91,10.04,10.06,9.76,1
1,0.0,-0.01,-0.01,0.05,-80.306057,26.920794,2088,-13.7,2.2,0.0,-0.28,0.04,0.17,7.85,10.85,8.88,8.60,1
2,0.0,-0.01,-0.02,0.10,-80.306057,26.920794,2112,-13.7,2.2,0.0,-0.36,-0.94,-0.04,7.64,9.06,8.24,9.06,1
3,0.0,-0.01,0.00,0.15,-80.306057,26.920794,2028,-13.6,2.3,0.0,0.09,0.24,-0.13,8.67,8.82,9.02,9.30,1
4,0.0,-0.01,0.01,0.20,-80.306057,26.920794,2148,-13.5,2.3,0.0,0.08,0.24,-0.04,13.50,7.96,8.90,9.13,1


In [49]:
pip2 = Pipeline([("test",separateDataFrames(df_id = "id"))])
col2 = pip2.transform(col)

In [50]:
col2[0]

,sideSlip,vxCG,vyCG,time,longitude,latitude,engineSpeed,handwheelAngle,throttle,brake,axCG,ayCG,yawRate,chassisAccelFL,chassisAccelFR,chassisAccelRL,chassisAccelRR
0,0.0,0.0,-0.0,0.00,NaN,NaN,210,-22.3,0.5,0.0,0.04,-0.20,-0.21,9.56,9.61,9.01,9.14
1,0.0,0.0,-0.0,0.05,NaN,NaN,210,-22.2,0.5,0.0,-0.09,-0.03,-0.03,9.58,9.64,9.16,9.25
2,0.0,0.0,-0.0,0.10,NaN,NaN,210,-22.3,0.5,0.0,-0.03,-0.10,0.11,9.60,9.61,9.07,9.14
3,0.0,0.0,-0.0,0.15,NaN,NaN,210,-22.2,0.5,0.0,-0.02,-0.09,0.11,9.61,9.62,9.01,9.10
4,0.0,0.0,-0.0,0.20,NaN,NaN,210,-22.2,0.5,0.0,0.01,-0.14,-0.35,9.59,9.59,9.06,9.20
